# Задача 1 Применение метаклассов

Метаклассы позволяют настраивать создание классов в Python. Определив метакласс для класса, вы можете контролировать создание экземпляров класса, его атрибуты и поведение.

Напишите метакласс Python *AttrLoggingMeta*, который регистрирует каждый доступ к атрибуту или его изменение. В метаклассе должен быть переопределен метод `__new__`. В *AttrLoggingMeta* добавить методы по логированию доступа *log_access(name, value)*, чтению *log_read(name, value, instance)* и записи *log_write(name, value, instance)* атрибута класса.

Определите класс *LoggedClass*, используя *AttrLoggingMeta* в качестве его метакласса. Проверьте правильность реализации методов.


In [49]:
class AttrLoggingMeta(type):
    
    # Переопределение метода __new__ для регистрации доступа к атрибутам.
    def __new__(cls, name, bases, cls_dct):
        
        # Перебираем атрибуты класса
        for key, value in cls_dct.items():
            if not key.startswith('__'):
                cls_dct[key] = cls.log_access(key, value)   
        return super().__new__(cls, name, bases, cls_dct)
    
    @staticmethod
    def log_access(name, value):
        
        # Если атрибут является методом, обернем его для регистрации вызовов метода
        if callable(value):
            def wrapper(*args, **kwargs):
                print(f"Calling method {name}")
                return value(*args, **kwargs)
            return wrapper
        
        else:
            
            # Если атрибут является свойством в классе, инициируем операции чтения и записи.
            property_name = f"_{name}"
            def property_get(self):
                AttrLoggingMeta.log_read(name, value, self)
                return getattr(self, property_name, value)

            def property_set(self, diff_value):
                AttrLoggingMeta.log_write(name, diff_value, self)
                setattr(self, property_name, diff_value)

            return property(property_get, property_set)
    
    @staticmethod
    def log_read(name, value, instance):
        print(f"Reading attribute {name}")
        return value
    
    @staticmethod
    def log_write(name, value, instance):
        print(f"Writing attribute {name} with value {value}")
        instance.__dict__[name] = value # Обновили словарь экземпляра новым значением

# Определяем класс LoggedClass, используя AttrLoggingMeta в качестве его метакласса
class LoggedClass(metaclass=AttrLoggingMeta):
    
    custom_method = 42 # Определяем атрибут класса
    
    # Определяем метод класса other_custom_method
    def other_custom_method(self): 
        pass


instance = LoggedClass() # Создаем экземпляр LoggedClass
print(instance.custom_method) # Получаем доступ и выводим значение атрибута custom_method
instance.custom_method = 78 # Запись атрибута custom_method со значением 78
instance.other_custom_method() # Вызываем метод other_custom_method

Reading attribute custom_method
42
Writing attribute custom_method with value 78
Calling method other_custom_method


# Задача 2 Динамическое создание класса

Динамическая природа Python позволяет изменять классы и объекты во время выполнения программы. Вы можете добавлять, удалять или изменять атрибуты и методы классов и объектов динамически с помощью встроенных функций, таких как `'setattr', 'getattr' и 'delattr'`.

Напишите функцию Python *create_class_with_methods*, которая принимает имя класса, словарь атрибутов и словарь методов и возвращает динамически созданный класс с этими атрибутами и методами. Для создания класса использовать метод *type*.

In [51]:
# Функция для динамического создания класса с указанными атрибутами и методами

def create_class_with_methods(name, dict_attrs, dict_methods):
    # Объединяем атрибуты и методы в один словарь
    dict_attrs.update(dict_methods)
    return type(name, (object,), dict_attrs)

# Определим атрибуты для динамического класса
attributes = {
    'species': 'Human',
    'age': 25
}

# Определим методы для динамического класса
methods = {
    'greet': lambda self: f"Hello, I am a {self.species} and I am {self.age} years old."
}

# Создаем динамический класс, используя определенные атрибуты и методы
DynamicClass = create_class_with_methods('DynamicClass', attributes, methods)

# Test DynamicClass

instance = DynamicClass()
print(instance.greet())

Hello, I am a Human and I am 25 years old.


# Задача 3 Генерация кода

Функции Python `«exec» и «eval»` позволяют выполнять динамически сгенерированный код во время выполнения. Эту функцию можно использовать для создания шаблонов кода, анализа
предметно-ориентированных языков (DSL) или реализации инструментов для генерации кода.

Напишите функцию Python *generate_complex_function*, которая принимает имя функции, список имён параметров и тело функции в виде строк и возвращает динамически сгенерированную функцию.

In [54]:
# Функция для динамической генерации сложной функции

def generate_complex_function(name, params, body):
    
    params_str = ', '.join(params) # Параметры через запятую
    
    indented_body = body.replace('\n', '\n    ') # Отступы в теле
    
    func_code = f"def {name}({params_str}):\n    {indented_body}"  # Создаем строку кода функции
    
    exec(func_code) # Выполняет сгенерированный код функции.
    
    return locals()[name] # Сгенерированная функция извлекается из locals().


# Определим параметры и тело функции
function_name = 'complex_function'
parameters = ['x', 'y']
function_body = """
if x > y:
    return x - y
else:
    return y - x
"""

complex_func = generate_complex_function(function_name, parameters, function_body)

# Проверка
print(complex_func(10, 5))
print(complex_func(5, 10))

5
5
